In [1]:
# import numpy as np
# from scipy.integrate import solve_ivp

# # %% Pendulum equations
# def pendulum(t, y):
#     theta, omega = y
#     dtheta_dt = omega
#     domega_dt = -np.sin(theta)
#     return [dtheta_dt, domega_dt]

In [2]:
# ## Generating initial conditions by using one single trajectory
# from scipy.integrate import solve_ivp
# import numpy as np

# # Set the seed for reproducibility
# np.random.seed(0)

# # Parameters
# n_steps = 10000  # Number of steps
# dt = 0.5  # Time step

# # Uniformly sampling one initial condition within the specified ranges
# theta_initial = np.random.uniform(-np.pi, np.pi, 1)
# omega_initial = np.random.uniform(0, 15, 1)

# # Combining the initial condition into an array
# initial_condition = np.column_stack((theta_initial, omega_initial))

# # Time points where we want to evaluate
# t_eval = np.linspace(0, n_steps * dt, n_steps + 1)

# # Solving the ODE for the initial condition
# trajectories = np.empty((1, len(t_eval), 2))

# sol = solve_ivp(pendulum, [0, n_steps * dt], initial_condition[0], t_eval=t_eval)
# trajectories[0] = sol.y.T

# print(trajectories.shape) # (1, 10001, 2)



In [3]:
# ## Generating initial conditions by manually setting up the grid
# M1 = 100
# X1 = np.linspace(-np.pi, np.pi, M1+1)
# X1 = X1 + (X1[1] - X1[0]) / 2
# X1 = X1[:-1]
# X2 = np.linspace(-15, 15, M1)
# initial_conditions = np.array(np.meshgrid(X1[:-1], X2)).T.reshape(-1, 2)
# # Time points where we want to evaluate
# t_eval = np.linspace(0.000001, 0.5, 11)  # 100 snapshots
# # t_eval = [0.000001, 0.5]

# # %% Solving the ODE for each initial condition
# trajectories = np.empty((initial_conditions.shape[0], len(t_eval), 2))

# for i, ic in enumerate(initial_conditions):
#     sol = solve_ivp(pendulum, [0, 10], ic, t_eval=t_eval)
#     y = sol.y.T
#     # Normalizing the angle to be within -pi to pi
#     # y[:, 0] = (y[:, 0] + np.pi) % (2 * np.pi) - np.pi
#     trajectories[i] = y

In [4]:
# ## Generating initial conditions by uniform distribution
# num_points = 1000

# # Uniformly sampling within the specified ranges
# # Using np.random.uniform with appropriate bounds to exclude the interval endpoints
# theta_initial = np.random.uniform(-np.pi, np.pi, num_points)
# omega_initial = np.random.uniform(0, 15, num_points)

# # Combining the initial conditions into an array
# initial_conditions = np.column_stack((theta_initial, omega_initial))

# # Time points where we want to evaluate
# t_eval = np.linspace(0, 5, 11)  # 10 snapshots

# # %% Solving the ODE for each initial condition
# trajectories = np.empty((initial_conditions.shape[0], len(t_eval), 2))

# for i, ic in enumerate(initial_conditions):
#     sol = solve_ivp(pendulum, [0, 10], ic, t_eval=t_eval)
#     trajectories[i] = sol.y.T

In [5]:
# # %% Applying modulus operation to keep angles within -pi to pi
# for i in range(trajectories.shape[0]):
#     for j in range(trajectories.shape[1]):
#         theta = trajectories[i, j, 0]
#         # Normalize theta to be within -pi to pi
#         trajectories[i, j, 0] = np.arctan2(np.sin(theta), np.cos(theta))

In [6]:
# # %%
# # Extracting the first 9 snapshots for DATA_X
# DATA_X = trajectories[:, :-1, :]
# # Extracting the last 9 snapshots for DATA_Y
# DATA_Y = trajectories[:, 1:, :]
# # Reshaping DATA_X and DATA_Y to maintain the third dimension
# # The reshaped matrices will have shapes (m*n, p)
# DATA_X = DATA_X.reshape(-1, DATA_X.shape[2])
# DATA_Y = DATA_Y.reshape(-1, DATA_Y.shape[2])

In [7]:
# # %% Specifying the file path
# import scipy.io
# file_path = 'D:\\resDMD_data\\data_pendulum_100_i.mat'
# # Saving DATA_X and DATA_Y to a .mat file at the specified location
# scipy.io.savemat(file_path, {'DATA_X': DATA_X, 'DATA_Y': DATA_Y})

In [8]:
# # Find the maximum value of the first column
# max_value_first_column = DATA_X[:, 0].max()

# max_value_first_column

In [9]:
from connector_pendulum import connector_pendulum

solver_indices = [5,2,3,0,4,1]
n_psi_trains = [22]

for solver_index in solver_indices:
    for n_psi_train in n_psi_trains:
        connector_pendulum(solver_index, n_psi_train)